<img src="https://www.colorado.edu/rc/sites/default/files/page/logo.png"
     alt="Logo for Research Computing @ University of Colorado Boulder"
     width="400" />
# Parallel computation of $\pi$

In this example, each engine computes a unique estimate of pi.
The result is averaged across all engines in the cluster.

In [3]:
!ipcluster start -n 4 --daemonize

In [4]:
import ipyparallel
import random
import time
import numpy as np

def estimate_pi(n):
    count = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) <= 1:
            count += 1
    return 4.0*count/float(n)

## Serial calculation

In [5]:
print('\n\n\n')
print('Serial Estimation of Pi')
print('')

for i in range(8):
    nx = 10**i

    t0 = time.time()

    est_pi = estimate_pi(nx)

    t1 = time.time()
    tval = t1-t0

    msg = 'Estimation based on '+str(10**i)+' points: '
    tmsg = 'Calculation time (seconds) : '
    print(msg,est_pi,tmsg,tval)





Serial Estimation of Pi

Estimation based on 1 points:  4.0 Calculation time (seconds) :  1.5020370483398438e-05
Estimation based on 10 points:  3.2 Calculation time (seconds) :  1.1205673217773438e-05
Estimation based on 100 points:  3.36 Calculation time (seconds) :  7.390975952148438e-05
Estimation based on 1000 points:  3.256 Calculation time (seconds) :  0.0005979537963867188
Estimation based on 10000 points:  3.142 Calculation time (seconds) :  0.005872249603271484
Estimation based on 100000 points:  3.14152 Calculation time (seconds) :  0.055680036544799805
Estimation based on 1000000 points:  3.14016 Calculation time (seconds) :  0.4819221496582031
Estimation based on 10000000 points:  3.1420904 Calculation time (seconds) :  4.759747743606567


## Parallel calculation

In [6]:
rc=ipyparallel.Client(profile='default')
nengines = len(rc)
all_proc = rc[:]
all_proc.block = True
nengines

4

### Import modules on the engines

Each engine has its own namespace, so functions, variables ... need to be defined on those engines. We can do this with the parallel magic commands as show. Alternativley you can use `sync_imports`

In [7]:
with all_proc.sync_imports():
    import random

importing random on engine(s)


### Running the calculation in parallel

In [8]:
print('\n\n\n')
print('Parallel Estimation of Pi')
print('')

#Now in parallel
for i in range(2,8):
    nx = 10**i

    t0 = time.time()

    #[nx//nengines]*nengines creates a list of length nengines, where each element
    # has value nx//nengines.  Each process gets one element of this list 
    # and passes it to estimate_pi

    pi_estimates = all_proc.map_sync(estimate_pi, [nx//nengines]*nengines)

    est_pi = np.mean(pi_estimates)

    t1 = time.time()
    tval = t1-t0

    msg = 'Estimation based on '+str(10**i)+' points: '
    tmsg = 'Calculation time (seconds) : '
    print(msg,est_pi,tmsg,tval)







Parallel Estimation of Pi

Estimation based on 100 points:  3.4 Calculation time (seconds) :  0.02008986473083496
Estimation based on 1000 points:  3.16 Calculation time (seconds) :  0.014931917190551758
Estimation based on 10000 points:  3.1372 Calculation time (seconds) :  0.014739990234375
Estimation based on 100000 points:  3.13596 Calculation time (seconds) :  0.02659893035888672
Estimation based on 1000000 points:  3.143408 Calculation time (seconds) :  0.15084528923034668
Estimation based on 10000000 points:  3.1416148 Calculation time (seconds) :  1.4087960720062256


In [9]:
!ipcluster stop

2019-05-20 21:41:07.505 [IPClusterStop] Stopping cluster [pid=32652] with [signal=<Signals.SIGINT: 2>]
